In [1]:
import os
import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
from matplotlib import pyplot as plt
import plotly.express as px

In [2]:
ROOT_FOLDER = os.path.join(os.getcwd(), "plot_data")

# Robustness

In [21]:
FILENAME = "robustness_350_class_8_small.csv"
FOLDER_PATH = os.path.join(ROOT_FOLDER, "robustness_data")
CSV_PATH = os.path.join(FOLDER_PATH, FILENAME)

df = pd.read_csv(CSV_PATH)
total_shift = np.add(df["Shift_X"], df["Shift_Y"])

In [22]:
max_row = df.iloc[df['mIoU'].idxmax()]
hline_str = f"mIoU: {max_row['mIoU']}, Shift: {max_row['Shift_X']},{max_row['Shift_Y']}"
print(f"Best record: {max_row}")

Best record: Angle      -0.050
Shift_X   -60.000
Shift_Y    80.000
mIoU        0.844
Name: 1070, dtype: float64


In [23]:
fig = px.scatter_3d(df, x='Angle', y=total_shift, z='mIoU',
              color='mIoU', width=1200, height=800, labels={"y":"Total Shift"})
fig.show()

In [24]:
fig = px.scatter(df, x="Angle", y="mIoU", color=total_shift,
                 width=1200, height=600, labels={"color":"Total Shift"})
fig.update_xaxes(tickvals=df["Angle"])
fig.show()

# Threshold Plots

In [25]:
FILENAME = "th_argmax_500.csv"
FOLDER_PATH = os.path.join(ROOT_FOLDER, "threshold_data")
CSV_PATH = os.path.join(FOLDER_PATH, FILENAME)

df = pd.read_csv(CSV_PATH)

In [26]:
max_row = df.iloc[df['IoU'].idxmax()]
hline_str = f"IoU: {np.round(max_row['IoU'], 3)}, Th_Values: {max_row['Th_Value']}"
print(f"Best record: {max_row}")

Best record: Unnamed: 0    2.000000
Th_Value      0.200000
IoU           0.838491
Name: 2, dtype: float64


In [27]:
fig = px.line(df, x="Th_Value", y="IoU", width=1200, height=600, markers=True)
fig.update_xaxes(tickvals=df["Th_Value"])
fig.show()

# Sweep plot

In [12]:
FILENAME = "slice_var_no_norm_no_l1.csv"
FOLDER_PATH = os.path.join(ROOT_FOLDER, "sweep_data")
CSV_PATH = os.path.join(FOLDER_PATH, FILENAME)

df = pd.read_csv(CSV_PATH)

In [13]:
max_row = df.iloc[df['aug_iou_single'].idxmax()]
print(max_row)

Name                     classic-sweep-65
Runtime                              1200
Notes                                   -
Tags                                  NaN
lambda_L2                            1.55
lambda_tv                             0.2
aug_iou_multiple                 0.854415
aug_iou_single                   0.811561
standard_iou_multiple            0.872417
standard_iou_single              0.829022
Name: 0, dtype: object


In [14]:
fig = px.scatter(df, x="lambda_L2", y="lambda_tv", color="aug_iou_single",
                 width=1200, height=600)
fig.show()

In [31]:
sum = df["lambda_L2"] + df["lambda_tv"]
norm_lambda_L2 = df["lambda_L2"] / sum
norm_lambda_tv = df["lambda_tv"] / sum

fig = px.scatter(df, x=norm_lambda_L2, y=norm_lambda_tv, color="avg_aug_SR_iou",
                 width=1200, height=600, labels={"x":"Lambda L2", "y":"Lambda TV"})
fig.show()


In [6]:
FILENAME = "slice_var_no_norm_l1.csv"
FOLDER_PATH = os.path.join(ROOT_FOLDER, "sweep_data")
CSV_PATH = os.path.join(FOLDER_PATH, FILENAME)

df = pd.read_csv(CSV_PATH)

In [7]:
max_row = df.iloc[df['aug_iou_single'].idxmax()]
print(max_row)

Name                     major-sweep-57
Runtime                            1200
Notes                                 -
Tags                                NaN
lambda_L1                          1.15
lambda_L2                          0.05
lambda_tv                          0.55
aug_iou_multiple               0.856414
aug_iou_single                 0.814413
standard_iou_multiple          0.872417
standard_iou_single            0.829022
Name: 0, dtype: object


In [8]:
fig = px.scatter_3d(df, x='lambda_L1', y='lambda_L2', z='lambda_tv',
              color='aug_iou_single', width=1200, height=800)
fig.show()

In [35]:
fig = px.scatter(df, x="lambda_L2", y="lambda_tv", color="aug_iou_single",
                 width=1200, height=600)
fig.show()

# Dropout Test

In [41]:
FILENAME = "100_rep_500_samples_max.csv"
FOLDER_PATH = os.path.join(ROOT_FOLDER, "dropout_data")
CSV_PATH = os.path.join(FOLDER_PATH, FILENAME)

df = pd.read_csv(CSV_PATH)

In [42]:
max_row = df.iloc[df['dIoU'].idxmax()]
print(max_row)

Unnamed: 0    0.000000
D_value       0.050000
sIoU          0.829024
dIoU          0.828006
Name: 0, dtype: float64


In [43]:
fig = px.scatter(df, x="D_value", y="dIoU", width=1200, height=600)
fig.add_hline(y=0.829)
fig.show()